In [1]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta

import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
start_date = datetime.now() - timedelta(hours=96)
print(start_date.strftime("%Y-%m-%d"))

2024-08-01


In [4]:
end_date = datetime.now() - timedelta(hours=72)
print(end_date.strftime("%Y-%m-%d"))

2024-08-02


In [5]:
feature_view = fs.get_feature_view('nvidia_stocks_fv', 1)
feature_view.init_batch_scoring(training_dataset_version=1)

In [6]:
print(feature_view.get_batch_query())

WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`ticker` `ticker`, `fg1`.`ticker` `join_pk_ticker`, `fg1`.`date` `join_evt_date`, `fg0`.`sentiment` `sentiment`, RANK() OVER (PARTITION BY `fg1`.`ticker`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `mymlcourse_featurestore`.`nvidia_stock_1` `fg1`
INNER JOIN `mymlcourse_featurestore`.`news_sentiment_updated_1` `fg0` ON `fg1`.`ticker` = `fg0`.`ticker` AND `fg1`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`date` `date`, `right_fg0`.`ticker` `ticker`, `right_fg0`.`sentiment` `sentiment`
FROM right_fg0)


In [7]:
nvidia_df_b = feature_view.get_batch_data(start_time=start_date, end_time=end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.14s) 


In [8]:
nvidia_df_b.head()

,date,ticker,sentiment
0,2024-08-02 00:00:00+00:00,NVDA,0.061445


In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
#OneHotEncoding the nvidia_df_b column 'ticker'

ticker = nvidia_df_b[['ticker']]

# Initializing OneHotEncoder
encoder = OneHotEncoder()

# Fitting and transforming the 'ticker' column
ticker_encoded_test = encoder.fit_transform(ticker)

# Converting the encoded column into a DataFrame
ticker_encoded_df_test = pd.DataFrame(ticker_encoded_test.toarray(), columns=encoder.get_feature_names_out(['ticker']))

# Concatenating the encoded DataFrame with the original DataFrame
nvidia_df_b = pd.concat([nvidia_df_b, ticker_encoded_df_test], axis=1)

# Dropping the original 'ticker' column
nvidia_df_b.drop('ticker', axis=1, inplace=True)
# As X_train['date'] column exists and is in datetime format, we're converting it
nvidia_df_b['year'] = nvidia_df_b['date'].dt.year
nvidia_df_b['month'] = nvidia_df_b['date'].dt.month
nvidia_df_b['day'] = nvidia_df_b['date'].dt.day

# Dropping the original date column
nvidia_df_b.drop(columns=['date'], inplace=True)

# Converting dataframe to numpy array
nvidia_df_b_array = nvidia_df_b.to_numpy()

# Reshaping the array to have a shape suitable for LSTM
nvidia_df_b_array = np.expand_dims(nvidia_df_b_array, axis=1)

In [12]:
import joblib

the_model = mr.get_model("stock_pred_model", version=1)
model_dir = the_model.download()

model = joblib.load(model_dir + "/stock_prediction_model.pkl")

In [13]:
predictions = model.predict(nvidia_df_b_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step


In [14]:
predictions 

array([[2.2207463]], dtype=float32)

In [15]:
import numpy as np

# My predictions array
predictions = np.array(predictions, dtype=np.float32)

# Changing the format of the predicted value to correspond with format of "open"
predictions = predictions[0][0]*100
print(predictions)

222.07462787628174


In [16]:
nvidia_df_b['predictions'] = predictions.tolist()

In [17]:
# Assuming you have 'year', 'month', and 'day' columns in your DataFrame
nvidia_df_b['date'] = pd.to_datetime(nvidia_df_b[['year', 'month', 'day']])

# Now you can drop the 'year', 'month', and 'day' columns if you want
nvidia_df_b.drop(columns=['year', 'month', 'day'], inplace=True)

In [18]:
nvidia_df_b['date'] = pd.to_datetime(nvidia_df_b['date'])

In [19]:
nvidia_df_b

,sentiment,ticker_NVDA,predictions,date
0,0.061445,1.0,222.074628,2024-08-02


In [20]:
# Convert the encoded DataFrame back to numpy array
ticker_encoded_array = ticker_encoded_df_test.to_numpy()

# Inverse transform the encoded array to retrieve the original values
original_tickers = encoder.inverse_transform(ticker_encoded_array)

# Convert the original_tickers array to a DataFrame
original_tickers_df = pd.DataFrame(original_tickers, columns=['ticker'])

# Concatenate the original ticker column with the remaining columns from nvidia_df_b
nvidia_df_b = pd.concat([nvidia_df_b.drop(columns=['ticker_NVDA']), original_tickers_df], axis=1)

In [21]:
nvidia_df_b.head()

,sentiment,predictions,date,ticker
0,0.061445,222.074628,2024-08-02,NVDA


In [ ]:
#from sklearn.preprocessing import MinMaxScaler

# Flatten the list of lists into a single list
#flat_predictions_scaled = [item for sublist in predictions_scaled for item in sublist]

# Initialize the MinMaxScaler
#scaler = MinMaxScaler()

# Fit the scaler to the scaled predictions
#scaler.fit(flat_predictions_scaled)

# Inverse transform the scaled predictions to get the original values
#predictions_unscaled = scaler.inverse_transform(flat_predictions_scaled)

# Update the 'predictions' column with the unscaled values
#nvidia_df_b['predictions'] = predictions_unscaled

In [22]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.


In [23]:
results_fg = fs.get_or_create_feature_group(
    name= 'stock_prediction_results',
    version = 1,
    description = 'Predction of NVDA open stock price',
    primary_key = ['ticker'],
    event_time = ['date'],
    online_enabled = False,
)

2024-08-05 20:34:29,949 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [24]:
#Inserting the stock data into the stocks feature group
results_fg.insert(nvidia_df_b, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fg/1072467


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: stock_prediction_results_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/stock_prediction_results_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x24931705a10>, None)